<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# загрузим нужные библиотеки
import os
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# откроем файл
pth1 = '/content/drive/MyDrive/Colab Notebooks/машинное обучение для текстов_проект для викишоп/toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [4]:
# выведем первые 5 строк
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
# посмотрим информацию о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
# найдем уникальные значения для столбца "toxic", который будет у нас целевым признаком
data.toxic.value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

налицо дисбаланс классов

In [7]:
df = pd.DataFrame(data)

In [8]:
# переведем текст в нижний регистр
df['text'] = df['text'].str.lower()

Лемматизируем текст

In [9]:
# инициализируем Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
# напишем функцию для pos-тегов - маркировки частей речи
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
# Lemmatize corpus with the appropriate POS tag
# print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(lem(clear_text(corpus[0])))])

In [11]:
# напишем функцию для очистки и лемматизации текста с маркированными частями речи
def clear_text(text):
    reg = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = reg.split() 
    lemm = []
    # в цикле лемматизируем очищенный текст
    for i in range(len(clear)):
      lemm.append(lemmatizer.lemmatize(clear[i], get_wordnet_pos(clear[i])))
    return " ".join(lemm)

In [12]:
# применим функцию к столбцу "текст" и создадим столбец с лемматризированным текстом
%%time
df['lemmatized_text'] = df['text'].apply(clear_text)

CPU times: user 23min 12s, sys: 1min 23s, total: 24min 36s
Wall time: 25min 30s


In [13]:
# посмотрим, что получилось
df.head()

,text,toxic,lemmatized_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,d'aww! he matches this background colour i'm s...,0,d aww he match this background colour i m seem...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not try to edit war it s ju...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"you, sir, are my hero. any chance you remember...",0,you sir be my hero any chance you remember wha...


### Вывод 1.

Мы получили датасет из 160 строк. Очистили его, разметили части речи и лемматизировали, т.е. привели слова к начальным формам. Создали столбец с лемматизированным текстом. Проверили целевой признак на дисбаланс классов и обнаружили дисбаланс. При обучении моделей будем это учитывать.

## Обучение

In [14]:
# разделим выборки на обучающую, валидационную и тестовую в пропорции 80%-10%-10%
target = df['toxic']
features = df['lemmatized_text']

#указываем stratify=target, чтобы сохранить соотношение классов при разделении
#сначала разделим на обучающую и тестовую в соотношении 80%-20%
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345, stratify=target) 

#разделим тестовую выборку пополам на тестовую и валидационную
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.50, random_state=12345, stratify=target_test)

In [15]:
# проверим размер выборок
display(features_train.shape)
display(target_train.shape)
display(features_valid.shape)
display(target_valid.shape)
display(features_test.shape)
display(target_test.shape)

(127656,)

(127656,)

(15957,)

(15957,)

(15958,)

(15958,)

In [16]:
# проверим выборки на дисбаланс классов
print(target_train.value_counts(
    normalize=True).mul(100).round(1).astype(str) + '%')
print(target_valid.value_counts(
    normalize=True).mul(100).round(1).astype(str) + '%')
print(target_test.value_counts(
    normalize=True).mul(100).round(1).astype(str) + '%')

0    89.8%
1    10.2%
Name: toxic, dtype: object
0    89.8%
1    10.2%
Name: toxic, dtype: object
0    89.8%
1    10.2%
Name: toxic, dtype: object


In [17]:
# найдём стоп-слова, то есть слова без смысловой нагрузки
# для этого вызовем функцию stopwords.words(), передадим ей аргумент
# 'english' для английского языка
stopwords = set(nltk_stopwords.words('english'))

# при создании счётчика передадим список стоп-слов в счётчик векторов CountVectorizer():
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [18]:
# Переведём тексты в стандартный для Python формат: кодировку Unicode U
train_corpus = features_train.values.astype('U')
valid_corpus = features_valid.values.astype('U')
test_corpus = features_test.values.astype('U')

# переведем слова в векторы и посчитаем tf_idf
'''Оценка важности слова определяется величиной TF-IDF (от англ. term frequency, 
«частота термина, или слова»; inverse document frequency, «обратная частота документа, или текста»). 
То есть TF отвечает за количество упоминаний слова в отдельном тексте, 
а IDF отражает частоту его употребления во всём корпусе.'''
train_tf_idf = count_tf_idf.fit_transform(train_corpus)
valid_tf_idf = count_tf_idf.transform(valid_corpus)
test_tf_idf = count_tf_idf.transform(test_corpus)

In [19]:
print("Размер матрицы:", train_tf_idf.shape)
print("Размер матрицы:", valid_tf_idf.shape)
print("Размер матрицы:", test_tf_idf.shape)

Размер матрицы: (127656, 132342)
Размер матрицы: (15957, 132342)
Размер матрицы: (15958, 132342)


In [ ]:
# pd.DataFrame(test_tf_idf.toarray()) это убило мое ядро

Для определения тональности применим величины TF-IDF как признаки.
Анализ тональности текста, или сентимент-анализ (от англ. sentiment, «настроение»), выявляет эмоционально окрашенные слова. Этот инструмент помогает компаниям оценивать, например, реакцию на запуск нового продукта в интернете. На разбор тысячи отзывов человек потратит несколько часов, а компьютер — пару минут.
Оценить тональность — значит отметить текст как позитивный или негативный. То есть мы решаем задачу классификации, где целевой признак равен «1» для положительного текста и «0» для отрицательного. Признаки — это слова из корпуса и их величины TF-IDF для каждого текста.

### 2.1 Логистическая регрессия

In [20]:
# обучим модель логистической регрессии, 
# укажем гиперпараметр class_weight='balanced'
%%time
model = LogisticRegression(class_weight='balanced', 
                          random_state=12345, 
                          max_iter=1000, 
                          solver='lbfgs')
# обучим модель на тренировочной выборке
model.fit(train_tf_idf, target_train)
# найдем предсказания на валидационной выборке
predicted_valid = model.predict(valid_tf_idf)
# напечатаем значение метрики F1
print("F1_score:", f1_score(target_valid, predicted_valid))

F1_score: 0.7617495245857104
CPU times: user 8.43 s, sys: 5.63 s, total: 14.1 s
Wall time: 9.06 s


### 2.2 Решающее дерево

In [21]:
# обучим модель решающего дерева для классификации и подберем наилучшую глубину
%%time
for depth in range(2, 20, 2):
    #создадим модель, указав max_depth=depth
    model = DecisionTreeClassifier(random_state=12345, 
                                  max_depth=depth,
                                  class_weight='balanced')
    #обучим модель
    model.fit(train_tf_idf, target_train)
    predicted_valid = model.predict(valid_tf_idf)
    print('Глубина:', depth)
    print("F1_score:", f1_score(target_valid, predicted_valid))    
    print('')
print()

Глубина: 2
F1_score: 0.3704443334997504

Глубина: 4
F1_score: 0.4762773722627737

Глубина: 6
F1_score: 0.5556492411467117

Глубина: 8
F1_score: 0.5741587037806397

Глубина: 10
F1_score: 0.6001596169193933

Глубина: 12
F1_score: 0.6124620060790275

Глубина: 14
F1_score: 0.6151537884471118

Глубина: 16
F1_score: 0.606694560669456

Глубина: 18
F1_score: 0.6144745998608212


CPU times: user 1min 25s, sys: 193 ms, total: 1min 25s
Wall time: 1min 28s


### 2.3 Случайный лес

In [43]:
# построим модель случайного леса для регрессии с подбором наилучших гиперпараметров
# найдем наилучшую глубину дерева
%%time
for depth in range(3, 20, 3):
    model = RandomForestClassifier(random_state=12345, 
                                   max_depth = depth, 
                                   class_weight='balanced')
    model.fit(train_tf_idf, target_train)
    predicted_valid = model.predict(valid_tf_idf)
    print('Глубина:', depth)
    print("F1_score:", f1_score(target_valid, predicted_valid))    
    print('')
print()

Глубина: 3
F1_score: 0.34397296282388284

Глубина: 6
F1_score: 0.33683701824332485

Глубина: 9
F1_score: 0.34502487562189055

Глубина: 12
F1_score: 0.35516404953402275

Глубина: 15
F1_score: 0.3659247482776894

Глубина: 18
F1_score: 0.3770603460019071


CPU times: user 46.3 s, sys: 705 ms, total: 47 s
Wall time: 47.5 s


Модель склонна к переобучению, поэтому ограничим глубину дерева 9

In [22]:
# найдем наилучшее количество деревьев
%%time
for est in [100, 500, 1000]:
    model = RandomForestClassifier(random_state=12345, 
                                   n_estimators=est, 
                                   max_depth=9,
                                   class_weight='balanced')
    model.fit(train_tf_idf, target_train)
    predicted_valid = model.predict(valid_tf_idf)
    print('Количество деревьев:', est)
    print("F1_score:", f1_score(target_valid, predicted_valid))    
    print('')
print()

Количество деревьев: 100
F1_score: 0.34502487562189055

Количество деревьев: 500
F1_score: 0.3510917030567686

Количество деревьев: 1000
F1_score: 0.3569706964353672


CPU times: user 1min 18s, sys: 1.04 s, total: 1min 19s
Wall time: 1min 19s


Лучший результат метрики F1 показала модель логистической регрессии. Проверим ее на тестовой выборке.

### 2.4 Проверка модели

In [23]:
model = LogisticRegression(class_weight='balanced', 
                          random_state=12345, 
                          max_iter=1000, 
                          solver='lbfgs')
# обучим модель на тренировочной выборке
model.fit(train_tf_idf, target_train)
# найдем предсказания на тестовой выборке
predicted_test = model.predict(test_tf_idf)
# выведем результат F1
print("F1_score:", f1_score(target_test, predicted_test))

F1_score: 0.7563716086599068


### Вывод 2. 

При обучении моделей наилучший результат показала модель логистической регрессии. Значение F1-меры на тестовой выборке - 0,75.
Сводная таблица моделей:

In [26]:
tabledata = [["логистическая регрессия", 0.75],
         ["решающее дерево", 0.60],
         ["случайный лес", 0.35]]
print("Обучение моделей для классифиции комментарии на позитивные и негативные")
df= pd.DataFrame(tabledata, columns=["модель","F1-мера"])
df = df.set_index('модель')
df.index.names = [None]
df

Обучение моделей для классифиции комментарии на позитивные и негативные


,F1-мера
логистическая регрессия,0.75
решающее дерево,0.60
случайный лес,0.35


## Выводы

1. Мы получили датасет из 160 строк. Очистили его, разметили части речи и лемматизировали, т.е. привели слова к начальным формам. Создали столбец с лемматизированным текстом. Проверили целевой признак на дисбаланс классов и обнаружили дисбаланс.

2. При обучении моделей с подбором гиперпараметров наилучший результат показала модель логистической регрессии. Значение F1-меры на тестовой выборке - 0,75.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны